### Imports

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', 500)

### Load Data

In [20]:
df = pd.read_csv('export_77721075/activities.csv')

In [21]:
df.head(3)

,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,Activity Private Note,Activity Gear,Filename,Athlete Weight,Bike Weight,Elapsed Time.1,Moving Time,Distance.1,Max Speed,Average Speed,Elevation Gain,Elevation Loss,Elevation Low,Elevation High,Max Grade,Average Grade,Average Positive Grade,Average Negative Grade,Max Cadence,Average Cadence,Max Heart Rate.1,Average Heart Rate,Max Watts,Average Watts,Calories,Max Temperature,Average Temperature,Relative Effort.1,Total Work,Number of Runs,Uphill Time,Downhill Time,Other Time,Perceived Exertion,"<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.type"">Type</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.start_time"">Start Time</span>",Weighted Average Power,Power Count,Prefer Perceived Exertion,Perceived Relative Effort,Commute.1,Total Weight Lifted,From Upload,Grade Adjusted Distance,Weather Observation Time,Weather Condition,Weather Temperature,Apparent Temperature,Dewpoint,Humidity,Weather Pressure,Wind Speed,Wind Gust,Wind Bearing,Precipitation Intensity,Sunrise Time,Sunset Time,Moon Phase,Bike,Gear,Precipitation Probability,Precipitation Type,Cloud Cover,Weather Visibility,UV Index,Weather Ozone,"<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.jump_count"">Jump Count</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_grit"">Total Grit</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_flow"">Avg Flow</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.flagged"">Flagged</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_elapsed_speed"">Avg Elapsed Speed</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.dirt_distance"">Dirt Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_distance"">Newly Explored Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_dirt_distance"">Newly Explored Dirt Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.sport_type"">Sport Type</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_steps"">Total Steps</span>",Media
0,4730876591,"Feb 4, 2021, 1:43:38 AM",Recovery Day Easy Pace 5K,Run,NaN,2175,5.04,NaN,NaN,False,NaN,NaN,activities/4730876591.gpx,NaN,NaN,2175.0,2117.0,5045.600098,4.7,2.383373,86.398712,84.498703,43.400002,59.099998,24.4,0.037657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,394.959198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,0.0,38.0,0.0,NaN,1.0,5357.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,4741462342,"Feb 6, 2021, 1:46:19 AM",FARTLEK - 1 Min HARD & 2 Mins EASY,Run,NaN,1927,4.48,NaN,NaN,False,NaN,NaN,activities/4741462342.gpx,NaN,NaN,1927.0,1875.0,4482.899902,4.9,2.390880,88.878464,96.978500,43.799999,58.299999,18.4,-0.180687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,357.206757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,0.0,20.0,0.0,NaN,1.0,4845.399902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

### Date-Time Conversion 

In [22]:
df['Activity Date Local'] = pd.to_datetime(df['Activity Date'])
df['Activity Date'] = pd.to_datetime(df['Activity Date'])

In [23]:
df['Location'] = np.where(
    df['Activity Date Local'] < pd.to_datetime('2022-08-01'),
    'India',
    'USA'
)

In [24]:
df['Activity Date Local'] = np.where(
    df['Location'] == 'India',
    df['Activity Date Local'] - pd.Timedelta(hours = 5.5),
    df['Activity Date Local'] + pd.Timedelta(hours = 6)
)

In [25]:
df['Local Time'] = df['Activity Date Local'].dt.strftime('%I:%M:%p').astype(str)

In [26]:
df['Local Time']

0      08:13:PM
1      08:16:PM
2      08:14:PM
3      08:17:PM
4      08:19:PM
         ...   
259    02:18:AM
260    12:40:AM
261    01:31:AM
262    01:55:AM
263    07:31:AM
Name: Local Time, Length: 264, dtype: object

### Activity Type

Group Hikes along with Walk

In [27]:
df['Activity Type'].value_counts()

Run                204
Ride                42
Walk                12
Hike                 5
Weight Training      1
Name: Activity Type, dtype: int64

In [28]:
df.loc[df['Activity Type'] == 'Hike', 'Activity Type'] = 'Walk'

In [29]:
df = df[df['Activity Type'].isin(['Run', 'Ride', 'Walk'])].copy()

### Speed and Time Metric Conversion

* m/sec to min/km = 16.67 Divided by the m/s value
* m/sec to kmph = Multiply by 3.6

In [30]:
df['Average Speed - OG'] = df['Average Speed'].copy()

df['Average Speed'] = np.where(
    df['Activity Type'] == 'Ride',
    np.floor(df['Average Speed'] * 3.6).astype(int).astype(str) + ' KMPH',
    np.floor(16.67/df['Average Speed']).astype(int).astype(str) + ' Min/KM'
)

In [31]:
df['Max Speed'].fillna(0, inplace=True)
df['Max Speed - OG'] = df['Max Speed'].copy()

df['Max Speed'] = np.where(
    df['Activity Type'] == 'Ride',
    np.floor(df['Max Speed'] * 3.6).astype(int).astype(str) + ' KMPH',
    np.floor(16.67/(df['Max Speed']+0.001)).astype(int).astype(str) + ' Min/KM'
)

Convert moving time from seconds to minutes

In [32]:
df['Moving Time - Minutes'] = df['Moving Time']/60

### Save New File

In [33]:
df.to_csv('activities_cleaned_may2023.csv', index=False)

In [34]:
df.head()

,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,Activity Private Note,Activity Gear,Filename,Athlete Weight,Bike Weight,Elapsed Time.1,Moving Time,Distance.1,Max Speed,Average Speed,Elevation Gain,Elevation Loss,Elevation Low,Elevation High,Max Grade,Average Grade,Average Positive Grade,Average Negative Grade,Max Cadence,Average Cadence,Max Heart Rate.1,Average Heart Rate,Max Watts,Average Watts,Calories,Max Temperature,Average Temperature,Relative Effort.1,Total Work,Number of Runs,Uphill Time,Downhill Time,Other Time,Perceived Exertion,"<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.type"">Type</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.start_time"">Start Time</span>",Weighted Average Power,Power Count,Prefer Perceived Exertion,Perceived Relative Effort,Commute.1,Total Weight Lifted,From Upload,Grade Adjusted Distance,Weather Observation Time,Weather Condition,Weather Temperature,Apparent Temperature,Dewpoint,Humidity,Weather Pressure,Wind Speed,Wind Gust,Wind Bearing,Precipitation Intensity,Sunrise Time,Sunset Time,Moon Phase,Bike,Gear,Precipitation Probability,Precipitation Type,Cloud Cover,Weather Visibility,UV Index,Weather Ozone,"<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.jump_count"">Jump Count</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_grit"">Total Grit</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_flow"">Avg Flow</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.flagged"">Flagged</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.avg_elapsed_speed"">Avg Elapsed Speed</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.dirt_distance"">Dirt Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_distance"">Newly Explored Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.newly_explored_dirt_distance"">Newly Explored Dirt Distance</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.sport_type"">Sport Type</span>","<span class=""translation_missing"" title=""translation missing: en-US.lib.export.portability_exporter.activities.horton_values.total_steps"">Total Steps</span>",Media,Activity Date Local,Location,Local Time,Average Speed - OG,Max Speed - OG,Moving Time - Minutes
0,4730876591,2021-02-04 01:43:38,Recovery Day Easy Pace 5K,Run,NaN,2175,5.04,NaN,NaN,False,NaN,NaN,activities/4730876591.gpx,NaN,NaN,2175.0,2117.0,5045.600098,3 Min/KM,6 Min/KM,86.398712,84.498703,43.400002,59.099998,24.400000,0.037657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,394.959198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,0.0,38.0,0.0,NaN,1.0,5357.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2021-02-03 20:13:38,India,08:13:PM,2.383373,4.7,35.283333
1,4741462342,2021-02-06 01:46:19,FARTLEK - 1 Min HARD & 2 Mins EASY,Run,NaN,1927,4.48,NaN,NaN,False,NaN,NaN,activities/4741462342.gpx,NaN,NaN,1927.0,1875.0,4482.899902,3 Min/KM,6 Min/KM,88.878464,96.978500,43.799999,58.299999,18.400000,-0.180